In [ ]:
%load_ext autoreload
%autoreload 2

import os
import sys

module_path = os.path.abspath(os.path.join('../..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from generator import RoadNetwork
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import pandas as pd
import networkx as nx
import numpy as np
from tqdm import tqdm

from models import SRN2VecModel
import json

In [ ]:
network = RoadNetwork()
network.load("../../osm_data/porto")

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = SRN2VecModel(None, device, network)

In [ ]:
model.generate_data(n_shortest_paths=100, number_negative=3, window_size=900, save_batch_size=100)

In [ ]:
%lprun -f model.extract_pairs model.generate_train_pairs(network, paths, 900, 7)

In [ ]:
with open("srn2vec-traindata.json", "r") as fp:
    a = json.load(fp)

In [37]:
"""
Build and train model
"""
network = RoadNetwork()
network.load("../../osm_data/porto")
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = SRN2VecModel(None, device, network)
model.load_dataset("./srn2vec-traindata.json")

In [38]:
model.train(epochs=5, batch_size=256)

Epoch: 0, Iteration: 0, sample_loss: 0.7239810228347778, Avg. Loss: 0.7239810228347778
Epoch: 0, Iteration: 1000, sample_loss: 0.5584360361099243, Avg. Loss: 0.615629504908334
Epoch: 0, Iteration: 2000, sample_loss: 0.5639439225196838, Avg. Loss: 0.5965400131448158
Epoch: 0, Iteration: 3000, sample_loss: 0.6287178993225098, Avg. Loss: 0.5890835301155489
Epoch: 0, Iteration: 4000, sample_loss: 0.5672624111175537, Avg. Loss: 0.5853222552402947
Epoch: 0, Iteration: 5000, sample_loss: 0.5696268081665039, Avg. Loss: 0.5832011812962287
Epoch: 0, Iteration: 6000, sample_loss: 0.5925624370574951, Avg. Loss: 0.5818337734699885
Epoch: 0, Iteration: 7000, sample_loss: 0.5950089693069458, Avg. Loss: 0.5806902458396678
Epoch: 0, Iteration: 8000, sample_loss: 0.5632270574569702, Avg. Loss: 0.579303998724995
Epoch: 0, Iteration: 9000, sample_loss: 0.5652724504470825, Avg. Loss: 0.5768814296930873
Epoch: 0, Iteration: 10000, sample_loss: 0.516416072845459, Avg. Loss: 0.5721476304359453
Epoch: 0, Itera

In [39]:
model.save_model(path="../model_states/srn2vec/")